In [2]:
import pandas as pd

import numpy as np

In [3]:
url = 'https://raw.github.com/pandas-dev/pandas/master/pandas/tests/data/tips.csv'

tips = pd.read_csv(url)
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [4]:
tips.shape

(244, 7)

# SELECT

In SQL, selection is done using a comma-separated list of columns you’d like to select (or a * to select all columns):

    SELECT total_bill, tip, smoker, time
    FROM tips
    LIMIT 5;

With pandas, column selection is done by passing a list of column names to your DataFrame:

In [5]:
columns = ['total_bill', 'tip', 'smoker', 'time']
tips[columns].head(5)

,total_bill,tip,smoker,time
0,16.99,1.01,No,Dinner
1,10.34,1.66,No,Dinner
2,21.01,3.50,No,Dinner
3,23.68,3.31,No,Dinner
4,24.59,3.61,No,Dinner


# WHERE

Filtering in SQL is done via a WHERE clause.

    SELECT *
    FROM tips
    WHERE time = 'Dinner'
    LIMIT 5;

DataFrames can be filtered in multiple ways; the most intuitive of which is using boolean indexing.

In [6]:
tips[tips['time'] == 'Dinner'].head(5)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


The above statement is simply passing a Series of True/False objects to the DataFrame, returning all rows with True.

In [7]:
is_dinner = tips['time'] == 'Dinner'

is_dinner.value_counts()

True     176
False     68
Name: time, dtype: int64

In [8]:
tips[is_dinner].head(5)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


Just like SQL’s OR and AND, multiple conditions can be passed to a DataFrame using | (OR) and & (AND).

    -- tips of more than $5.00 at Dinner meals
    SELECT *
    FROM tips
    WHERE time = 'Dinner' AND tip > 5.00;



In [9]:
tips[(tips['time'] == 'Dinner') & (tips['tip'] > 5.00)]

,total_bill,tip,sex,smoker,day,time,size
23,39.42,7.58,Male,No,Sat,Dinner,4
44,30.40,5.60,Male,No,Sun,Dinner,4
47,32.40,6.00,Male,No,Sun,Dinner,4
52,34.81,5.20,Female,No,Sun,Dinner,4
59,48.27,6.73,Male,No,Sat,Dinner,4
116,29.93,5.07,Male,No,Sun,Dinner,4
155,29.85,5.14,Female,No,Sun,Dinner,5
170,50.81,10.00,Male,Yes,Sat,Dinner,3
172,7.25,5.15,Male,Yes,Sun,Dinner,2
181,23.33,5.65,Male,Yes,Sun,Dinner,2


    -- tips by parties of at least 5 diners OR bill total was more than $45
    SELECT *
    FROM tips
    WHERE size >= 5 OR total_bill > 45;

In [10]:
tips[(tips['size'] >= 5) | (tips['total_bill'] > 45)]

,total_bill,tip,sex,smoker,day,time,size
59,48.27,6.73,Male,No,Sat,Dinner,4
125,29.80,4.20,Female,No,Thur,Lunch,6
141,34.30,6.70,Male,No,Thur,Lunch,6
142,41.19,5.00,Male,No,Thur,Lunch,5
143,27.05,5.00,Female,No,Thur,Lunch,6
155,29.85,5.14,Female,No,Sun,Dinner,5
156,48.17,5.00,Male,No,Sun,Dinner,6
170,50.81,10.00,Male,Yes,Sat,Dinner,3
182,45.35,3.50,Male,Yes,Sun,Dinner,3
185,20.69,5.00,Male,No,Sun,Dinner,5


NULL checking is done using the notnull() and isnull() methods.

In [11]:
frame = pd.DataFrame({'col1': ['A', 'B', np.NaN, 'C', 'D'],
                      'col2': ['F', np.NaN, 'G', 'H', 'I']})
frame

,col1,col2
0,A,F
1,B,NaN
2,NaN,G
3,C,H
4,D,I


Assume we have a table of the same structure as our DataFrame above. We can see only the records where col2 IS NULL with the following query:

    SELECT *
    FROM frame
    WHERE col2 IS NULL;



In [12]:
frame[frame['col2'].isnull()]

,col1,col2
1,B,NaN


Getting items where col1 IS NOT NULL can be done with notnull().

    SELECT *
    FROM frame
    WHERE col1 IS NOT NULL;



In [13]:
frame[frame['col1'].notnull()]

,col1,col2
0,A,F
1,B,NaN
3,C,H
4,D,I


# GROUP BY

In pandas, SQL’s GROUP BY operations are performed using the similarly named groupby() method. groupby() typically refers to a process where we’d like to split a dataset into groups, apply some function (typically aggregation) , and then combine the groups together.

A common SQL operation would be getting the count of records in each group throughout a dataset. For instance, a query getting us the number of tips left by sex:

    SELECT sex, count(*)
    FROM tips
    GROUP BY sex;
    /*
    Female     87
    Male      157
    */

The pandas equivalent would be:

In [14]:
tips.groupby('sex').size()

sex
Female     87
Male      157
dtype: int64

Notice that in the pandas code we used size() and not count(). This is because count() applies the function to each column, returning the number of not null records within each.

In [15]:
tips.groupby('sex').count()

,total_bill,tip,smoker,day,time,size
sex,,,,,,
Female,87,87,87,87,87,87
Male,157,157,157,157,157,157


Multiple functions can also be applied at once. For instance, say we’d like to see how tip amount differs by day of the week - agg() allows you to pass a dictionary to your grouped DataFrame, indicating which functions to apply to specific columns.

    SELECT day, AVG(tip), COUNT(*)
    FROM tips
    GROUP BY day;
    /*
    Fri   2.734737   19
    Sat   2.993103   87
    Sun   3.255132   76
    Thur  2.771452   62
    */



In [16]:
tips.groupby('day').agg({'tip': np.mean, 'day': np.size})

,tip,day
day,,
Fri,2.734737,19
Sat,2.993103,87
Sun,3.255132,76
Thur,2.771452,62


Grouping by more than one column is done by passing a list of columns to the groupby() method.

    SELECT smoker, day, COUNT(*), AVG(tip)
    FROM tips
    GROUP BY smoker, day;
    /*
    smoker day
    No     Fri      4  2.812500
           Sat     45  3.102889
           Sun     57  3.167895
           Thur    45  2.673778
    Yes    Fri     15  2.714000
           Sat     42  2.875476
           Sun     19  3.516842
           Thur    17  3.030000
    */



In [17]:
tips.groupby(['smoker', 'day']).agg({'tip': [np.size, np.mean]})

tip          
             size      mean
smoker day                 
No     Fri    4.0  2.812500
       Sat   45.0  3.102889
       Sun   57.0  3.167895
       Thur  45.0  2.673778
Yes    Fri   15.0  2.714000
       Sat   42.0  2.875476
       Sun   19.0  3.516842
       Thur  17.0  3.030000

# JOIN

JOINs can be performed with join() or merge(). By default, join() will join the DataFrames on their indices. Each method has parameters allowing you to specify the type of join to perform (LEFT, RIGHT, INNER, FULL) or the columns to join on (column names or indices).

In [18]:
df1 = pd.DataFrame({'key': ['A', 'B', 'C', 'D'],
                    'value': np.random.randn(4)})
df2 = pd.DataFrame({'key': ['B', 'D', 'D', 'E'],
                    'value': np.random.randn(4)})

In [19]:
df1

,key,value
0,A,1.124025
1,B,0.904185
2,C,-0.272988
3,D,0.619634


In [20]:
df2

,key,value
0,B,0.681934
1,D,1.440887
2,D,-0.339308
3,E,0.751939


Assume we have two database tables of the same name and structure as our DataFrames.

Now let’s go over the various types of JOINs.
## INNER JOIN

    SELECT *
    FROM df1
    INNER JOIN df2
      ON df1.key = df2.key;



In [21]:
pd.merge(df1, df2, on='key')

,key,value_x,value_y
0,B,0.904185,0.681934
1,D,0.619634,1.440887
2,D,0.619634,-0.339308


merge() also offers parameters for cases when you’d like to join one DataFrame’s column with another DataFrame’s index.

In [22]:
indexed_df2 = df2.set_index('key')
indexed_df2

,value
key,
B,0.681934
D,1.440887
D,-0.339308
E,0.751939


In [23]:
pd.merge(df1, indexed_df2, left_on='key', right_index=True)

,key,value_x,value_y
1,B,0.904185,0.681934
3,D,0.619634,1.440887
3,D,0.619634,-0.339308


## LEFT OUTER JOIN

    -- show all records from df1
    SELECT *
    FROM df1
    LEFT OUTER JOIN df2
      ON df1.key = df2.key;



In [24]:
pd.merge(df1, df2, on='key', how='left')

,key,value_x,value_y
0,A,1.124025,NaN
1,B,0.904185,0.681934
2,C,-0.272988,NaN
3,D,0.619634,1.440887
4,D,0.619634,-0.339308


## RIGHT JOIN¶

    -- show all records from df2
    SELECT *
    FROM df1
    RIGHT OUTER JOIN df2
      ON df1.key = df2.key;



In [25]:
 pd.merge(df1, df2, on='key', how='right')

,key,value_x,value_y
0,B,0.904185,0.681934
1,D,0.619634,1.440887
2,D,0.619634,-0.339308
3,E,NaN,0.751939


FULL JOIN

pandas also allows for FULL JOINs, which display both sides of the dataset, whether or not the joined columns find a match. As of writing, FULL JOINs are not supported in all RDBMS (MySQL).

    -- show all records from both tables
    SELECT *
    FROM df1
    FULL OUTER JOIN df2
      ON df1.key = df2.key;



In [26]:
pd.merge(df1, df2, on='key', how='outer')

,key,value_x,value_y
0,A,1.124025,NaN
1,B,0.904185,0.681934
2,C,-0.272988,NaN
3,D,0.619634,1.440887
4,D,0.619634,-0.339308
5,E,NaN,0.751939


# UPDATE

    UPDATE tips
    SET tip = tip*2
    WHERE tip < 2;



In [27]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [28]:
tips.loc[tips['tip'] < 2, 'tip'] = tips.loc[tips['tip'] < 2, 'tip']* 2
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,2.02,Female,No,Sun,Dinner,2
1,10.34,3.32,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


# DELETE

    DELETE FROM tips
    WHERE tip > 9;

In pandas we select the rows that should remain, instead of deleting them

In [29]:
tips.shape

(244, 7)

In [30]:
tips.loc[tips['tip'] >  9]

,total_bill,tip,sex,smoker,day,time,size
170,50.81,10.0,Male,Yes,Sat,Dinner,3


In [31]:
tips_2 = tips.loc[tips['tip'] <= 9]
tips_2.shape

(243, 7)

In [32]:
tips_2.loc[tips['tip'] >  9]

,total_bill,tip,sex,smoker,day,time,size
